## Operator overloading

Warning: Advanced Topic!

### Operator overloading

We need to use a metaprogramming trick to make this teaching notebook work.
I want to be able to put explanatory text in between parts of a class definition,
so I'll define a decorator to help me build up a class definition gradually.

In [352]:
def extend(class_to_extend):
    """ Metaprogramming to allow gradual implementation
    of class during notebook. Thanks to
    http://www.ianbicking.org/blog/2007/08/opening-python-classes.html """
    def decorator(extending_class):
        for name, value in extending_class.__dict__.items():
            if name in ['__dict__','__module__', '__weakref__', '__doc__']:
                continue
            setattr(class_to_extend,name,value)
        return class_to_extend
    return decorator





Imagine we wanted to make a library to describe some kind of symbolic algebra system:




In [353]:
class Term(object):
    def __init__(self, symbols=[], powers=[], coefficient=1):
        self.coefficient = coefficient
        self.data={symbol: exponent for symbol,exponent
                in zip(symbols, powers)}

In [354]:
class Expression(object):
    def __init__(self, terms):
        self.terms=terms




So that $5x^2y+7x+2$ might be constructed as:




In [355]:
first=Term(['x','y'],[2,1],5)

second=Term(['x'],[1],7)

third=Term([],[],2)

result=Expression([first, second, third])




This is pretty cumbersome.

What we'd really like is to have `2x+y` give an appropriate expression.

First, we'll define things so that we can construct our terms and expressions in different ways.




In [356]:
class Term(object):
    def __init__(self, *args):
        lead=args[0]
        if type(lead)==type(self):
            # Copy constructor
            self.data = dict(lead.data)
            self.coefficient = lead.coefficient
        elif type(lead) == int:
            self.from_constant(lead)
        elif type(lead) == str:
            self.from_symbol(*args)
        elif type(lead) == dict:
            self.from_dictionary(*args)
        else:
            self.from_lists(*args)
            
    def from_constant(self, constant):
        self.coefficient = constant
        self.data={}
        
    def from_symbol(self, symbol, coefficient=1, power=1):
        self.coefficient = coefficient
        self.data={symbol:power}
        
    def from_dictionary(self, data, coefficient=1):
        self.data = data
        self.coefficient = coefficient
        
    def from_lists(self, symbols=[], powers=[], coefficient=1):
        self.coefficient=coefficient
        self.data={symbol: exponent for symbol,exponent
                in zip(symbols, powers)}

In [357]:
class Expression(object):
    def __init__(self, terms=[]):
        self.terms = list(terms)




We could define add() and multiply() operations on expressions and terms:




In [397]:
print(not(True or True))

False


In [403]:
@extend(Term)
class Term(object):
    def add(self, *others):
        #result_data=dict(self.data)
        #result_coeff = self.coefficient
        # my code
        added = self
        for another in others:
            if not (isinstance(another, Term) or isinstance(another, Expression)):
                another = Term(another)
            if isinstance(added, Term):
                if dict(added.data)==another.data:
                    added = Term(dict(added.data), added.coefficient+another.coefficient)
                else:
                    added = Expression((added,)+(another,))
            else:
                eq_term = list(filter(lambda x: another.data == x.data, added.terms))
                other_terms = tuple(list(filter(lambda x: another.data != x.data, added.terms)))
                if len(eq_term) > 0:
                    eq_term = Term(dict(another.data), eq_term[0].coefficient+another.coefficient)
                    added = Expression((eq_term,)+other_terms)
                else:
                    added = Expression(other_terms+(another,))
        return added
        # my code
        # return Expression((self,)+others)
    

In [359]:
@extend(Term)
class Term(object):
    def multiply(self, *others):
        result_data=dict(self.data)
        result_coeff = self.coefficient
        # Convert arguments to Terms first if they are
        # constants or integers
        others=map(Term,others)
        
        for another in others:
            for symbol, exponent in another.data.items():
                if symbol in result_data:
                    result_data[symbol] += another.data[symbol]
                else:
                    result_data[symbol] = another.data[symbol]
            result_coeff*=another.coefficient
        
        return Term(result_data,result_coeff)

In [360]:
@extend(Expression)
class Expression(object):
    def add(self, *others):
        result = Expression(self.terms)
        
        for another in others:
            if type(another)==Term:
                result.terms.append(another)
            else:
                result.terms+=another.terms
                
        return result




We can now construct the above expression as:




In [361]:
x=Term('x')
y=Term('y')

first=Term(5).multiply(Term('x'),Term('x'),Term('y'))
second=Term(7).multiply(Term('x'))
third=Term(2)
expr=first.add(second,third)

In [362]:
print(expr.terms[0].data)
print(expr.terms[0].coefficient)
print(expr.terms[1].data)
print(expr.terms[1].coefficient)
print(expr.terms[2].data)
print(expr.terms[2].coefficient)

{'x': 2, 'y': 1}
5
{'x': 1}
7
{}
2


In [363]:
some_term = Term(['x'],[1],5).add(Term('x'),Term(['z'],[2],5),Term(['x'],[1],2))
#print(some_term.data)
#print(some_term.coefficient)
print(some_term.terms[0].data)
print(some_term.terms[0].coefficient)
print(some_term.terms[1].data)
print(some_term.terms[1].coefficient)
#print(some_term.terms[2].data)
#print(some_term.terms[2].coefficient)

{'x': 1}
8
{'z': 2}
5


# 

This is better, but we still can't write the expression in a 'natural' way.

However, we can define what `*` and `+` do when applied to Terms!:




In [384]:
@extend(Term)
class Term(object):
    
    def __add__(self, other):
        return self.add(other)
    
    def __mul__(self, other):
        return self.multiply(other)

In [385]:
@extend(Expression)
class Expression(object):
    def multiply(self, another):
        # Distributive law left as exercise
        pass
    
    def __add__(self, other):
        return self.add(other)

In [402]:
x_plus_y=Term('x')+'y'
#x_plus_y=Term('x')+'y'
x_plus_y.terms[1].data

{'y': 1}

In [367]:
five_x_ysq=Term('x')*5*'y'*'y'

print(five_x_ysq.data, five_x_ysq.coefficient)

{'x': 1, 'y': 2} 5





This is called operator overloading. We can define what add and multiply mean when applied to our class.

Note that this only works so far if we multiply on the right-hand-side!
However, we can define a multiplication that works backwards, which is used as a fallback if the left multiply raises an error:




In [368]:
@extend(Expression)
class Expression(object):
    def __radd__(self, other):
        return self.__add__(other)

In [369]:
@extend(Term)
class Term(object):
    def __rmul__(self, other):
        return self.__mul__(other)
    
    def __radd__(self, other):
        return self.__add__(other)

In [370]:
5*Term('x')




It's not easy at the moment to see if these things are working!




In [371]:
fivex=5*Term('x')
fivex.data, fivex.coefficient

({'x': 1}, 5)




We can add another operator method `__str__`, which defines what happens if we try to print our class:




In [372]:
@extend(Term)
class Term(object):
    def __str__(self):
        def symbol_string(symbol, power):
            if power==1:
                return symbol
            else:
                return symbol+'^'+str(power)
            
        symbol_strings=[symbol_string(symbol, power)
                for symbol, power in self.data.items()]
        
        prod='*'.join(symbol_strings)
        
        if not prod:
            return str(self.coefficient)
        if self.coefficient==1:
            return prod
        else:
            return str(self.coefficient)+'*'+prod

In [373]:
@extend(Expression)
class Expression(object):
    def __str__(self):
        return '+'.join(map(str,self.terms))

In [377]:
first=Term(5)*'x'*'x'*'y'
second=Term(7)*'x'
third=Term(2)
expr=first+second+third

In [375]:
print(expr)

5*x^2*y+7*x+2


We can add lots more operators to classes. `__eq__` to determine if objects are
equal. `__getitem__` to apply [1] to your object. Probably the most exciting
one is `__call__`, which overrides the `()` operator; allows us to define classes that *behave like
functions*! We call these callables.

In [ ]:
class Greeter(object):
    def __init__(self, greeting):
        self.greeting = greeting
        
    def __call__(self, name):
        print(self.greeting, name)

greeter_instance = Greeter("Hello")

greeter_instance("James")




We've now come full circle in the blurring of the distinction between functions and objects! The full power of functional programming is really remarkable.

If you want to know more about the topics in this lecture, using a different
language syntax, I recommend you watch the [Abelson and Sussman](https://www.youtube.com/watch?v=2Op3QLzMgSY)
"Structure and Interpretation of Computer Programs" lectures. These are the Computer Science
equivalent of the Feynman Lectures!
